In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

## Reading in Dataset

### Fixed Cost of each WTP

In [4]:
fixed_cost = pd.read_excel("../dataset2/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Fixed Cost
WTP,
Ampang Intake,11638
Batang Kali,11011
Bernam River Head,11602
WTP1,4899
WTP2,11773


### Capacity of each WTP

In [5]:
capacity = pd.read_excel("../dataset2/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,2.641160e+07
Batang Kali,2.858907e+07
Bernam River Head,2.993616e+07
WTP1,2.728402e+07
WTP2,2.898516e+07


In [6]:
capacity.loc["WTP1", "Capacity"]

27284019.78365

### Distribution Loss

In [7]:
distribution_loss = pd.read_excel("../dataset2/distribution_loss.xlsx", index_col=0)
distribution_loss.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.31838,0.34819,0.23241,0.26049,0.34462,0.28837,0.30351,0.28379,0.33333,0.30272,...,0.29068,0.28830,0.29829,0.35956,0.26697,0.30082,0.27045,0.28345,0.27237,0.33081
DMZ002,0.30544,0.32145,0.29041,0.26520,0.30354,0.25078,0.31171,0.26908,0.24993,0.34083,...,0.31445,0.24484,0.35810,0.34966,0.31344,0.25027,0.29574,0.27309,0.32620,0.30501
DMZ003,0.32782,0.29775,0.30906,0.33927,0.31701,0.29337,0.26027,0.31059,0.30448,0.32909,...,0.24614,0.32459,0.23055,0.31726,0.31519,0.31940,0.29601,0.28883,0.33544,0.26115
DMZ004,0.35452,0.28941,0.25286,0.30884,0.32873,0.27913,0.26008,0.33860,0.32277,0.28385,...,0.32659,0.26638,0.28412,0.34979,0.26636,0.28850,0.28361,0.29359,0.25466,0.26176
DMZ005,0.28498,0.28067,0.29266,0.32907,0.29612,0.33973,0.24868,0.34067,0.25028,0.28086,...,0.31274,0.31326,0.29827,0.27726,0.29009,0.23659,0.27612,0.35060,0.32065,0.27252


In [8]:
distribution_loss.loc["DMZ001", "WTP1"]

0.26049

### Links between WTP and DMZ

In [9]:
linkage = pd.read_excel("../dataset2/linkage.xlsx", index_col=0)
linkage.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
DMZ002,1,1,1,0,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
DMZ003,1,1,1,0,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
DMZ004,1,1,1,1,1,1,1,1,0,1,...,1,0,1,1,1,1,1,1,1,1
DMZ005,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1


### Variable Cost for each combination of (DMZ, WTP)

In [10]:
variable_cost = pd.read_excel("../dataset2/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,9,14,9,12,24,13,21,20,8,10,...,21,23,11,9,17,7,19,18,5,22
DMZ002,12,15,23,11,18,23,17,7,7,22,...,9,9,10,23,13,18,9,15,9,11
DMZ003,5,20,19,8,9,17,7,12,18,9,...,21,21,19,15,15,5,6,16,11,21
DMZ004,22,14,15,6,12,8,9,19,17,11,...,15,9,9,24,20,8,19,13,15,5
DMZ005,22,10,7,21,22,12,21,20,9,18,...,8,14,18,23,20,14,7,5,6,5


### Transport Cost for each combination of (DMZ, WTP)

In [11]:
# transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
# transport_cost.head()

### Demand for each DMZ

In [12]:
demand = pd.read_excel("../dataset2/demand_chisq.xlsx", index_col=0)
demand.head()

,Demand
DMZ001,160044.62961
DMZ002,150056.74766
DMZ003,141891.56807
DMZ004,194644.00762
DMZ005,126901.55527


In [13]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [68]:
# demand["Demand"] = pd.to_numeric(demand["Demand"], downcast="float")
# demand.head()

## Optimisation

In [14]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [15]:
print(wtp[:5])
print("Number of WTP:", len(wtp) )

print(dmz[:5])
print("Number of DMZ:", len(dmz) )

print("Number of DMZ-WTP Pairs:", len(dmz_wtp_pairs))

['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']
Number of WTP: 45
['DMZ001', 'DMZ002', 'DMZ003', 'DMZ004', 'DMZ005']
Number of DMZ: 2500
Number of DMZ-WTP Pairs: 112500


In [16]:
demand.loc[dmz[0], "Demand"]

160044.62961

In [17]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

/Users/daniel/opt/anaconda3/envs/walmart2/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [223]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [73]:
# fixed_cost.loc[wtp[0], "Fixed Cost"]

In [126]:
output[("DMZ001", "WTP1")]

Volume_('DMZ001',_'WTP1')

In [224]:
# Define the Objective Function

## Without transport cost
model += \
     lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])


## With transport cost
# model += \
#      lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
#      lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [128]:
curr_dmz=dmz[1]
curr_wtp=wtp[0]

In [129]:
linkage.loc[curr_dmz, curr_wtp]

1

In [130]:
1- distribution_loss.loc[curr_dmz, curr_wtp]

0.6945600000000001

In [131]:
output[(curr_dmz, curr_wtp)] * (1- distribution_loss.loc[curr_dmz, curr_wtp]) * (linkage.loc[curr_dmz, curr_wtp])

0.6945600000000001*Volume_('DMZ002',_'Ampang_Intake') + 0.0

In [79]:
# type(demand.loc["DMZ1", "Demand"])
# type(1-distribution_loss.loc["DMZ1", wtp[0]])

In [211]:
capacity.loc["Ampang Intake", "Capacity"]

26411595.83813

In [225]:
distribution_loss.loc["DMZ001", "Ampang Intake"]

0.31838

In [227]:
# # Adding Constraints

# ## Meet demand for each DMZ
# for d in dmz:
#     model += lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"]

# ## Within the WTP capacity
# for w in wtp:
#     model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"]

# ## No linkage constraint

# Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += LpConstraint(
        lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]),
        sense=LpConstraintGE,
        rhs= demand.loc[d, "Demand"],
        name=d
        )

## Within the WTP capacity
for w in wtp:
    model += LpConstraint(
        lpSum([output[(d, w)] for d in dmz]),
        sense = LpConstraintLE,
        rhs = capacity.loc[w, "Capacity"],
        name = w.replace(" ", "")
        )

## No linkage constraint



In [228]:
# Solve the model
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/daniel/opt/anaconda3/envs/walmart2/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/gp/1_dqbwsn08sbtf7s0k5q2lsc0000gn/T/e4f791a5d86047b0a9733bdcac4db4d1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/gp/1_dqbwsn08sbtf7s0k5q2lsc0000gn/T/e4f791a5d86047b0a9733bdcac4db4d1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2550 COLUMNS
At line 317559 RHS
At line 320105 BOUNDS
At line 320106 ENDATA
Problem MODEL has 2545 rows, 112500 columns and 202508 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2545 (0) rows, 90008 (-22492) columns and 180016 (-22492) elements
Perturbing problem by 0.001% of 24 - largest nonzero change 0.00018199579 ( 0.0017397892%) - largest zero change 0
0  Obj 0 Primal inf 4.1126425e+08 (2500)
125  Obj 2.2021697e+08 Primal inf 3.80

1

In [229]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 3,361,819,321 ($/Month)

Status: Optimal


In [230]:
soln_dict = {i.name: i.varValue for i in model.variables()}

In [248]:
not_satisfied = []

for c in model.constraints.values():
    c_dict = c.toDict()
    # print(c_dict)

    satisfied = False
    
    LHS = sum([soln_dict[i['name']]*i['value'] for i in c_dict['coefficients']])
    LHS = LHS + c_dict['constant']
    
    if c_dict['sense'] == 0:
        satisfied = (LHS == 0)
   
    if c_dict['sense'] == -1:
        satisfied = (LHS <= 0)
    
    if c_dict['sense'] == 1:
        satisfied = (LHS >= 0)
   
    # print(c)
    
    if satisfied:
        #print('LHS: ', LHS)
        print('is satisfied')
    else:
        #print('LHS: ', LHS)
        print(c_dict["name"],'not satisfied', "LHS:", LHS)
        not_satisfied.append(c_dict["name"])

DMZ001 not satisfied LHS: -0.0026547000161372125
is satisfied
DMZ003 not satisfied LHS: -0.002938400022685528
DMZ004 not satisfied LHS: -0.003575199982151389
DMZ005 not satisfied LHS: -0.0022683999850414693
is satisfied
DMZ007 not satisfied LHS: -0.0008851999882608652
DMZ008 not satisfied LHS: -0.0014620000147260725
DMZ009 not satisfied LHS: -0.0033360000234097242
is satisfied
DMZ011 not satisfied LHS: -0.001102800015360117
is satisfied
is satisfied
DMZ014 not satisfied LHS: -0.002233799983514473
is satisfied
DMZ016 not satisfied LHS: -0.0014420000370591879
DMZ017 not satisfied LHS: -0.002502000017557293
is satisfied
is satisfied
DMZ020 not satisfied LHS: -0.00326250001671724
is satisfied
is satisfied
DMZ023 not satisfied LHS: -0.0005476000078488141
is satisfied
is satisfied
is satisfied
DMZ027 not satisfied LHS: -0.0013524999958463013
is satisfied
DMZ029 not satisfied LHS: -0.0026080000097863376
DMZ030 not satisfied LHS: -0.0010919999913312495
is satisfied
DMZ032 not satisfied LHS: -0

In [258]:
"DMZ107" in not_satisfied

True

## Results

In [235]:
dict_wtp = {}
dict_dmz = {}

In [236]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [237]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    curr_dmz = combi[0]
    curr_wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[curr_dmz, curr_wtp] = volume


In [238]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WTP1,WTP10,WTP11,WTP12,WTP13,WTP14,...,WTP38,WTP39,WTP4,WTP40,WTP5,WTP6,WTP7,WTP8,WTP9,WangsaMaju
DMZ001,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,219953.31
DMZ002,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ003,211091.62,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ004,0.00,0.0,0.0,263659.52,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ005,0.00,0.0,0.0,174439.92,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ995,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ996,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,209710.92
DMZ997,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,338351.29,0.00
DMZ998,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00


In [ ]:
# df.columns = ["Ampang Intake", "Batang Kali", "Bername River Head", "Sungai Tengi", "Wangsa Maju"]
# df = df[["Ampang Intake", "Batang Kali", "Bername River Head", "Wangsa Maju", "Sungai Tengi", ]]

In [87]:
# Supply of water from each WTP to each DMZ (including distribution loss)
# df

### Renaming the columns for distribution loss to match result df

In [239]:
col_list = df.columns.tolist()
print(col_list[:5])

temp_col_list = distribution_loss.columns.tolist()
dist_col_list = [w.replace(" ", "") for w in temp_col_list]

print(dist_col_list[:5])

['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'SungaiTengi', 'WTP1']
['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'WTP1', 'WTP2']


In [240]:
result_distribution_loss = distribution_loss.copy()
result_distribution_loss.columns = dist_col_list

In [241]:
result_distribution_loss = result_distribution_loss[col_list]

### Accounting for distribution loss

In [249]:
useful_amount = 1- result_distribution_loss.values

In [250]:
temp = df.values * useful_amount
temp

array([[     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       , 160044.6269553],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       [141891.5651316,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       ...,
       [     0.       ,      0.       ,      0.       , ...,
             0.       , 238869.2437142,      0.       ],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ]])

In [251]:
temp.shape

(2500, 45)

In [252]:
# Supply of water from each WTP to each DMZ, after accounting for water loss
final_df = pd.DataFrame(temp, columns = wtp, index=dmz)
final_df.round(3)

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,160044.627
DMZ002,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ003,141891.565,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ004,0.000,0.0,0.0,194644.004,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ005,0.000,0.0,0.0,126901.553,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ2496,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ2497,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,149676.975
DMZ2498,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,238869.244,0.000
DMZ2499,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000


In [93]:
# wtp

In [253]:
supply = final_df.sum(axis=1)
supply[:5]

DMZ001    160044.626955
DMZ002    150056.750404
DMZ003    141891.565132
DMZ004    194644.004045
DMZ005    126901.553002
dtype: float64

In [254]:
sum(final_df.loc["DMZ001",:])

160044.6269553

In [179]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [255]:
prob = []
for curr_dmz in dmz:
    # print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
    # print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
        print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
        prob.append(curr_dmz)

DMZ101 217483.380066 >= 135604.55233 ?
DMZ102 171713.56738769996 >= 219335.13647 ?
DMZ103 160445.5190752 >= 162679.28167 ?
DMZ104 149853.56038799998 >= 147445.34278 ?
DMZ105 168450.9581226 >= 192934.71108 ?
DMZ106 166118.1801 >= 145223.87737 ?
DMZ107 136591.2520794 >= 203053.81513 ?
DMZ108 156857.0332595 >= 151811.03598 ?
DMZ109 183772.21423 >= 160503.88455 ?
DMZ110 150423.1229016 >= 167461.64841 ?
DMZ111 137289.8362689 >= 137247.72666 ?
DMZ112 208783.2044933 >= 134913.98211 ?
DMZ113 140725.6186812 >= 170376.29337 ?
DMZ114 149467.339819 >= 200656.84275 ?
DMZ115 133120.6889312 >= 211408.31361 ?
DMZ116 160021.3342745 >= 161936.34138 ?
DMZ117 141376.1620642 >= 176386.39911 ?
DMZ118 139785.8113263 >= 148959.6222 ?
DMZ119 203300.3817525 >= 184917.60021 ?
DMZ120 173602.939472 >= 174459.2079 ?
DMZ121 138163.51112580003 >= 233416.07347 ?
DMZ122 214317.708506 >= 214621.31332 ?
DMZ123 158390.176092 >= 206664.34339 ?
DMZ124 133138.496659 >= 190825.24332 ?
DMZ125 140871.021453 >= 168214.77578 ?
DM

In [185]:
len(prob)

2400

In [ ]:
final_df.sum(axis=0)

Ampang Intake        1.491667e+06
Batang Kali          4.878317e+05
Bernam River Head    1.500000e+06
Wangsa Maju          0.000000e+00
Sungai Tengi         1.415502e+06
dtype: float64

In [ ]:
demand.loc["DMZ5", "Demand"]

160000.0